In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd

import wfdb
import wfdb.processing as wp
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import signal
import shutil

import time
from scipy import sparse
import os
import warnings
warnings.filterwarnings("ignore")
import scipy.io as sio
import pywt

In [2]:
path = "/media/jdcloud/"
labels = pd.read_csv(path+"REFERENCE.csv")
labels.head()

,File_name,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10
0,TRAIN0000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRAIN0001,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRAIN0002,4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TRAIN0003,6,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRAIN0004,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
def read_data_labels_info(data_path, split = "Train",preprocess=True):
    """ Read data """

    # Paths
    path_signals = os.path.join(data_path, split)

    # Read labels and one-hot encode
    labels = pd.read_csv(data_path+"REFERENCE.csv")
    #2156+224+672+654+180+826+534+504+1953
    
    # Read time-series data
    channel_files = os.listdir(path_signals)
    channel_files.sort()
    n_channels = 12#len(channel_files)

    # Initiate array
    list_of_channels = []
    i_ch = 0
    
    data_x = []
    data_y = []
    
    channel_name = ['V6', 'aVF', 'I', 'V4', 'V2', 'aVL', 'V1','II', 'aVR', 'V3', 'III', 'V5']
    channel_mid_name = ['II','aVR','V2','V5']
    channel_post_name = ['III','aVF','V3','V6']
      
    info = []
    
    for i_ch,fil_ch in enumerate(channel_files[:]):#tqdm

        labels_list = list(labels.iloc[i_ch].values[:])#.dropna()
        #print(labels_list)

        ecg = sio.loadmat(os.path.join(path_signals,fil_ch))
        a = 0
        for m in labels_list:
            if m in [0,1,2,3,4,5,6,7,8,9]:
                a += 1
        labels_list.append(a)
        labels_list.append(ecg['sex'][0])
        labels_list.append(ecg['age'][0][0])
        labels_list.append(ecg['I'].shape[1])
        
        
        info.append(labels_list)
    column_name = ['File_name','label1','label2','label3','label4','label5','label6',
                   'label7','label8','label9','label10','labels_num','sex','age','dataSize']
    info_pd =pd.DataFrame(columns=column_name,index=range(len(channel_files)),data=info)
    # Return 
    return info_pd

In [4]:
data_info = read_data_labels_info(path)

In [5]:
data_info.head(10) 

,File_name,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,labels_num,sex,age,dataSize
0,TRAIN0000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,39,5000
1,TRAIN0001,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,36,5000
2,TRAIN0002,4,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE,80,5000
3,TRAIN0003,6,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE,74,5000
4,TRAIN0004,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,69,5000
5,TRAIN0005,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,26,7500
6,TRAIN0006,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE,64,5000
7,TRAIN0007,3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE,72,5000
8,TRAIN0008,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,49,5000
9,TRAIN0009,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE,79,5000


In [11]:
#8 - 7 - 6 - 5 - 4 - 3 - 2 - 1 - 0
2156+224+672+654+180+826+534+504+1953

7703

In [11]:
data_info[(data_info.label1==7) & (data_info.labels_num==1)]['label7'].index.shape

(194,)

In [72]:
data_info[((data_info.labels_num ==4))]

,File_name,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,labels_num,sex,age,dataSize
198,TRAIN0198,2,5.0,6.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,87,15000
666,TRAIN0666,5,4.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,81,5000
1217,TRAIN1217,5,4.0,3.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,50,5000
1374,TRAIN1374,5,1.0,8.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4,FEMALE,76,5000
1713,TRAIN1713,1,4.0,8.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,80,5000
2432,TRAIN2432,3,4.0,2.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,73,5000
2733,TRAIN2733,5,4.0,3.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,76,5000
3670,TRAIN3670,5,4.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,94,5000
3767,TRAIN3767,2,4.0,3.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,83,7500
4072,TRAIN4072,4,3.0,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4,MALE,90,8500


In [16]:
from collections import Counter

In [23]:
data_info['label1'].values

array([0, 0, 4, ..., 0, 2, 0])

In [27]:
Counter(data_info['label1'].values)

Counter({0: 1953,
         1: 406,
         2: 414,
         3: 704,
         4: 101,
         5: 502,
         6: 532,
         7: 204,
         8: 1473,
         9: 400})

In [81]:
l = []
l.append(dict(Counter(data_info['label1'].values)))
l.append(dict(Counter(data_info['label2'].values)))
l.append(dict(Counter(data_info['label3'].values)))
l.append(dict(Counter(data_info['label4'].values)))
l.append(dict(Counter(data_info['label5'].values)))
l.append(dict(Counter(data_info['label6'].values)))

for num in range(0,10):
    count = 0
    for i in range(6):
        try:
            count += l[i][num]
        except KeyError:
            count += 0
    print("num {0}  : {1}".format(num,count))


num 0  : 1953
num 1  : 478
num 2  : 492
num 3  : 926
num 4  : 164
num 5  : 627
num 6  : 628
num 7  : 222
num 8  : 2049
num 9  : 400


In [82]:
# 0   1   2   3   4   5   6   7   8    9
1953+478+492+926+164+627+628+222+2049+400

7939

In [77]:
num = 4
dict(Counter(data_info['label1'].values))[num]

101

In [40]:
dict(Counter(data_info['label2'].values))[8]

503

In [14]:
data_info[(data_info.label1==7) & (data_info.labels_num==4)]['label7'].index.shape

(0,)

In [75]:
data_info[data_info.label1==7]['label7'].index.shape

(205,)

In [73]:
data_info[data_info.label2==7]['label7'].index.shape

(19,)

In [69]:
data_info[data_info.label1==4]['label4'].index.shape

(111,)

In [70]:
data_info[data_info.label2==4]['label4'].index.shape

(66,)

In [71]:
data_info[data_info.label3==4]['label4'].index.shape

(3,)

In [8]:
data_info[data_info.labels_num>1]['label1'].index.shape

(1035,)

In [63]:
data_info[data_info.label1==0]['label1'].index.shape

(1953,)

In [64]:
data_info[data_info.label1==8]['label8'].index.shape

(1547,)

In [65]:
data_info[data_info.label2==8]['label8'].index.shape

(532,)

In [66]:
data_info[data_info.label3==8]['label8'].index.shape

(71,)

In [9]:
data_info['labels_num'].value_counts()

1    5639
2     869
3     165
4      14
6       1
5       1
Name: labels_num, dtype: int64

In [10]:
5639+2*869+3*165+4*14+6+5

7939

In [14]:
5465+2*881+3*143+4*9+11

7703

In [123]:
data_info[(data_info.dataSize<4500)]['dataSize'].value_counts()

Series([], Name: dataSize, dtype: int64)

In [85]:
label2_list = data_info[data_info.labels_num==2].index.tolist()
pd2 = data_info.loc[label2_list]
pd2.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

In [86]:
label3_list = data_info[data_info.labels_num==3].index.tolist()
pd3_1 = data_info.loc[label3_list]
pd3_1.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd3_2 = data_info.loc[label3_list]
pd3_2.columns = ['File_name', 'label3', 'label2', 'label1', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

In [87]:
label4_list = data_info[data_info.labels_num==4].index.tolist()
pd4_1 = data_info.loc[label4_list]
pd4_1.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd4_2 = data_info.loc[label4_list]
pd4_2.columns = ['File_name', 'label3', 'label2', 'label1', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd4_3 = data_info.loc[label4_list]
pd4_3.columns = ['File_name', 'label4', 'label2', 'label3', 'label1', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

In [95]:
data_info.loc[data_info[data_info.labels_num==5].index.tolist()]

,File_name,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,labels_num,sex,age,dataSize
5925,TRAIN5925,2,8.0,3.0,6.0,5.0,NaN,NaN,NaN,NaN,NaN,5,MALE,83,19500


In [97]:
label5_list = data_info[data_info.labels_num==5].index.tolist()
pd5_1 = data_info.loc[label5_list]
pd5_1.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd5_2 = data_info.loc[label5_list]
pd5_2.columns = ['File_name', 'label3', 'label2', 'label1', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd5_3 = data_info.loc[label5_list]
pd5_3.columns = ['File_name', 'label4', 'label2', 'label3', 'label1', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd5_4 = data_info.loc[label5_list]
pd5_4.columns = ['File_name', 'label5', 'label2', 'label3', 'label4', 'label1', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

In [96]:
label6_list = data_info[data_info.labels_num==6].index.tolist()
data_info.loc[label6_list]

,File_name,label1,label2,label3,label4,label5,label6,label7,label8,label9,label10,labels_num,sex,age,dataSize
5921,TRAIN5921,2,4.0,6.0,8.0,3.0,5.0,NaN,NaN,NaN,NaN,6,FEMALE,86,14000


In [98]:
label6_list = data_info[data_info.labels_num==6].index.tolist()
pd6_1 = data_info.loc[label6_list]
pd6_1.columns = ['File_name', 'label2', 'label1', 'label3', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd6_2 = data_info.loc[label6_list]
pd6_2.columns = ['File_name', 'label3', 'label2', 'label1', 'label4', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd6_3 = data_info.loc[label6_list]
pd6_3.columns = ['File_name', 'label4', 'label2', 'label3', 'label1', 'label5', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd6_4 = data_info.loc[label6_list]
pd6_4.columns = ['File_name', 'label5', 'label2', 'label3', 'label4', 'label1', 'label6',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

pd6_5 = data_info.loc[label6_list]
pd6_5.columns = ['File_name', 'label6', 'label2', 'label3', 'label4', 'label5', 'label1',
       'label7', 'label8','label9', 'label10','labels_num','sex','age','dataSize']

In [107]:
labels_en = pd.concat([data_info,pd2,pd3_1,pd3_2,pd4_1,pd4_2,pd4_3,pd5_1,pd5_2,pd5_3,
                       pd5_4,pd6_1,pd6_2,pd6_3,pd6_4,pd6_5])#[:6510]
labels_en.sort_index(inplace=True)#.head(8)

In [108]:
labels_en

,File_name,age,dataSize,label1,label10,label2,label3,label4,label5,label6,label7,label8,label9,labels_num,sex
0,TRAIN0000,39,5000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
1,TRAIN0001,36,5000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
2,TRAIN0002,80,5000,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE
2,TRAIN0002,80,5000,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE
3,TRAIN0003,74,5000,6.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE
3,TRAIN0003,74,5000,3.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE
4,TRAIN0004,69,5000,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
5,TRAIN0005,26,7500,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,MALE
6,TRAIN0006,64,5000,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,FEMALE
7,TRAIN0007,72,5000,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,MALE


In [111]:
labels_en.to_csv("kfold_labels_en.csv")

In [110]:
labels_en[labels_en.labels_num == 3]["sex"].value_counts()

MALE      336
FEMALE    159
Name: sex, dtype: int64

In [112]:
os.getcwd()

'/media/uuser/data/finals'